# 環境設定

In [ ]:
import os
import sys

def _is_in_kaggle() -> bool:
  """Whether the current environment is in `Kaggle`."""
  return str(_dh[0]) == '/kaggle/working'


def _is_in_colab() -> bool:
  """Whether the current environment is in `Colab`."""
  return 'google.colab' in str(get_ipython())

In [ ]:
if _is_in_kaggle():
  print('in kaggle')

elif _is_in_colab():
  print('in colab')
  from google.colab import drive
  drive.mount('/content/drive')  # mount google drive
  DRIVE_ROOT = '/content/drive/MyDrive'
  sys.path.append('/content/drive/MyDrive/Colab Notebooks/my-modules')  # path from drive

else:
  print('in local')

in colab
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Modelの扱い

## scratchでのBert読み込み

Bertを例にTransformersでのModelの構築を行う。
まずはscratchでBertを学習させるという仮定のもとModelを呼び出す。

In [ ]:
from transformers import BertConfig, BertModel, BertTokenizer

# Configを使ったモデル呼び出し
config = BertConfig()
model = BertModel(config)

In [ ]:
# BertConfigには、BERTのネットワークを構成する為のパラメータが含まれている。
print(config)

BertConfig {
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.38.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}



In [ ]:
# BertModelのネットワーク詳細を表示
print(model)

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
  

このModelを学習させることにより、BERTを１から作ることができるが、これは非常に大きな計算リソースと時間を要することになり、無謀である。

そこで、既に学習済みのBERTモデルを呼び出し、このモデルを使うことで短時間にかつ安価に有用なモデルを作成することができる。

## 学習済みモデルの呼び出し

In [ ]:
model = BertModel.from_pretrained("bert-base-uncased")

## モデルの保存

作成したモデルを保存することもできる。

In [ ]:
# モデルを保存
model.save_pretrained("./bert_model")

In [ ]:
# configとモデルの重みが保存されている
ls ./bert_model

SyntaxError: invalid syntax (<ipython-input-45-b6125a41d908>, line 2)

In [ ]:
# 保存したモデルを読み込む
model_saved = BertModel.from_pretrained("./bert_model")
model_saved

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
  

# 推論

機械学習モデルとは、つまるところテンソル(数字の配列や行列など)を入出力とする関数の近似である。
そのため、機械学習モデルの入力はテンソルである必要があると言える。

Bertも同様で、入力となる文字列(のリスト)はTokenizerによりテンソルに変換して初めてBertに渡すことができる。

In [ ]:
sequences = ["Hello!", "Cool.", "Nice!"]

In [ ]:
# tokenizerを使って、文字列をID列に変換
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
encoded_sequences = tokenizer(sequences).input_ids  # 通常はinput_idsのみを入力することは無いが、ここでは説明のため
encoded_sequences

[[101, 7592, 999, 102], [101, 4658, 1012, 102], [101, 3835, 999, 102]]

In [ ]:
# modelへの入力はtensorである必要がある
import torch
model_inputs = torch.tensor(encoded_sequences)

In [ ]:
#modelへ入力
model(model_inputs)

BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[-5.0937e-02,  1.0882e-01, -1.4107e-01,  ..., -1.2431e-01,
          -8.0330e-02,  2.8579e-01],
         [-6.7714e-01, -5.4644e-01,  8.7821e-02,  ..., -5.7526e-02,
           3.5945e-02, -3.0803e-01],
         [-1.0903e+00, -9.9962e-01, -5.6360e-01,  ...,  3.2317e-01,
          -2.7725e-01, -1.4627e-01],
         [ 8.3018e-01,  5.0071e-02, -2.2515e-01,  ...,  3.2161e-01,
          -6.4891e-01, -2.4565e-01]],

        [[-2.8582e-01, -1.8635e-02,  7.9945e-02,  ..., -3.2150e-01,
           3.1830e-01,  5.7760e-01],
         [ 4.5425e-01, -5.5394e-01,  3.9479e-01,  ..., -2.2183e-01,
           1.8811e-01,  1.0380e-01],
         [-2.7685e-01, -1.0924e+00,  2.2841e-01,  ...,  2.6004e-01,
           3.7289e-01,  6.6872e-02],
         [ 9.7986e-01, -2.8927e-02, -1.4129e-01,  ...,  3.6114e-01,
          -6.3582e-01, -1.5223e-01]],

        [[-7.1083e-04,  1.6308e-01, -1.2003e-01,  ..., -1.5940e-02,
          -5.3252e-02,  3